# Sendy Logistics Challenge

Sendy, in partnership with insight2impact facility, is hosting a Zindi challenge to predict the estimated time of delivery of orders, from the point of driver pickup to the point of arrival at final destination.

The solution will help Sendy enhance customer communication and improve the reliability of its service; which will ultimately improve customer experience. In addition, the solution will enable Sendy to realise cost savings, and ultimately reduce the cost of doing business, through improved resource management and planning for order scheduling.

## Import the Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

## Import the Data

In [ ]:
train = pd.read_csv('https://raw.githubusercontent.com/mhloti/Regress-Predict-Project/master/Datasets/Train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/mhloti/Regress-Predict-Project/master/Datasets/Test.csv')
riders = pd.read_csv('https://raw.githubusercontent.com/mhloti/Regress-Predict-Project/master/Datasets/Riders.csv')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
riders.head()

## Exploratory Data Analysis

## Date Preprocessing

## Model Performance

## Model Analysis